# Linear Optimization : Assignment 5

Name 1 : Ahmik Virani <br>
Roll Number 1 : ES22BTECH11001

Name 2 : Divya Rajparia <br>
Roll Number 2 : ES22BTECH11013

In [ ]:
import csv
import numpy as np
import pandas as pd
import heapq

df = pd.read_csv('Testcase.csv', header=None)
df = df.dropna(axis=1, how='all')

u = df.iloc[0, :].astype(int).to_list()
v = df.iloc[1, :].astype(int).to_list()
capacity = df.iloc[2, :].astype(float).to_list()
cost = df.iloc[3, :].astype(float).to_list()

no_of_edges = len(capacity)

for i in range(no_of_edges):
  print(f"Edge from {u[i]} to {v[i]} with capacity {capacity[i]} and cost {cost[i]}")

Edge from 1 to 2 with capacity 10.0 and cost 2.0
Edge from 1 to 3 with capacity 4.0 and cost 1.0
Edge from 1 to 4 with capacity 2.0 and cost 3.0
Edge from 3 to 2 with capacity 6.0 and cost 1.0
Edge from 4 to 5 with capacity 3.0 and cost 2.0


### Algorithm Flow

1. At each iteration, find the minimum-cost augmenting path from source to sink using reduced-cost Dijkstra.
2. Send as much flow as possible along this path, update residual capacities and reverse edges, and accumulate the total cost.
3. Repeat until no more augmenting paths exist.

First let us build the residual graph

In [ ]:
number_of_vertices = max(max(u), max(v))

# Create an adjacency matrix
adj = [[] for _ in range(number_of_vertices + 1)]

for i in range(no_of_edges):
  # Forward edge
  adj[u[i]].append([v[i], capacity[i], cost[i], len(adj[v[i]])])

  # Backward edge
  adj[v[i]].append([u[i], 0, -cost[i], len(adj[u[i]]) - 1])

print(adj)

[[], [[2, 10.0, 2.0, 0], [3, 4.0, 1.0, 0], [4, 2.0, 3.0, 0]], [[1, 0, -2.0, 0], [3, 0, -1.0, 1]], [[1, 0, -1.0, 1], [2, 6.0, 1.0, 1]], [[1, 0, -3.0, 2], [5, 3.0, 2.0, 0]], [[4, 0, -2.0, 1]]]


- Source vertex is 1
- Destination vertex is 2

In [ ]:
# Let us define dual potential for reduced cost
potential = [0] * (number_of_vertices + 1)

In [ ]:
source = 1
sink = 2

# We define a function to run dijkstra since we want minimum cost
def dijkstra():
  # Initially nothing is visited
  dist = [np.inf] * (number_of_vertices + 1)

  dist[source] = 0

  # Parent of each is not yet defined
  parent = [(-1, -1)] * (number_of_vertices + 1)

  priority_queue = [(0, source)] # (Reduced cost, node)

  while priority_queue:
    cur_dist, cur_vertex = heapq.heappop(priority_queue)

    if cur_dist > dist[cur_vertex]:
      continue

    # We check indexed from u->v and have capacity cap and cost c
    for idx, (v, cap, c,  _) in enumerate(adj[cur_vertex]):
      if cap > 0:
        reduced_cost = c + potential[cur_vertex] - potential[v]
        new_dist = cur_dist + reduced_cost

        if new_dist < dist[v]:
          dist[v] = new_dist
          parent[v] = (cur_vertex, idx)
          heapq.heappush(priority_queue, (new_dist, v))

  if dist[sink] == np.inf:
    return False, parent, dist

  # Update for all dual potentials
  for i in range(1, number_of_vertices + 1):
    if dist[i] < np.inf:
      potential[i] += dist[i]

  return True, parent, dist

Next we repetedly run bfs and compute the bottleneck

In [ ]:
total_flow = 0
iteration_no = 0
total_cost = 0

while True:
  # Check if min cost augmenting path exists
  found, parent, dist = dijkstra()
  if not found:
    print("No more augmenting paths")
    break

  # Find the bottleneck flow
  path_flow = np.inf
  cur = sink
  while cur != source:
    prev, index = parent[cur]
    path_flow = min(path_flow, adj[prev][index][1])
    cur = prev

  # Augment flow along the path and find cost
  path_cost = 0
  cur = sink
  while cur != source:
    prev, index = parent[cur]
    edge_cost = adj[prev][index][2]
    path_cost += edge_cost * path_flow

    # Update residual graph
    adj[prev][index][1] -= path_flow
    adj[cur][adj[prev][index][3]][1] += path_flow
    cur = prev

  total_flow += path_flow
  iteration_no+=1
  total_cost+=path_cost

  print(f"Iteration {iteration_no}: Total cost = {total_cost}")

print(f"\nFinal Max Flow = {total_flow}")
print(f"Final Min Cost = {total_cost}")

Iteration 1: Total cost = 20.0
Iteration 2: Total cost = 28.0
No more augmenting paths

Final Max Flow = 14.0
Final Min Cost = 28.0
